<img src="data/photutils_banner.svg">

## Photutils

- Code: https://github.com/astropy/photutils
- Documentation: http://photutils.readthedocs.org/en/latest/
- Issue Tracker:  https://github.com/astropy/photutils/issues

## Photutils Overview

- Background and background RMS estimation
- Aperture Photometry
- PSF Photometry (needs work)
- Source Detection and Extraction
  - DAOFIND and IRAF's starfind
  - image segmentation based on (local) threshold
  - local peak finder
- Segmentation-based source photometry and morphological properties
- Centroids
- More to come!

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['image.cmap'] = 'Greys_r'

# run the %matplotlib magic command to enable inline plotting
# in the current Notebook
%matplotlib inline

## Part 1:  Aperture Photometry

### Perform aperture photometry on some sources in the XDF

We'll start by reading the XDF WFC3/IR F160W data and error arrays from FITS files.

In [ ]:
from astropy.io import fits
sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'
sci_hdulist = fits.open(sci_fn)
rms_hdulist = fits.open(rms_fn)

In [ ]:
sci_hdulist.info()

In [ ]:
# define the data and error arrays
data = sci_hdulist[0].data.astype(np.float)
error = rms_hdulist[0].data.astype(np.float)
print(data.shape)
print(data.dtype)

In [ ]:
# extract the data header and create a WCS object
from astropy.wcs import WCS
hdr = sci_hdulist[0].header
wcs = WCS(hdr)

In [ ]:
# display the data
from astropy.visualization import scale_image
plt.imshow(scale_image(data, scale='sqrt', percent=99.5))

In [ ]:
# plot a histogram of data values
r = plt.hist(data.ravel(), bins=50, range=(-0.002, 0.005))

## Aperture photometry

<img src="data/photutils_apertures.svg">

## Methods for handling aperture/pixel intersection

<img src="data/photutils_aperture_methods.svg">

In [ ]:
from photutils import CircularAperture, aperture_photometry

In [ ]:
# define the aperture
position = (90.73, 59.43)
radius = 5.
aperture = CircularAperture(position, r=radius)

In [ ]:
# perform the photometry; the default method is 'exact'
phot = aperture_photometry(data, aperture)

In [ ]:
phot

In [ ]:
# center method
phot = aperture_photometry(data, aperture,  method='center')
phot

In [ ]:
# subpixel method, subpixels=5 (same as SExtractor)
phot = aperture_photometry(data, aperture,  method='subpixel', subpixels=5)
phot

In [ ]:
# input the error array to get the photometric error
phot = aperture_photometry(data, aperture, error=error)
phot

In [ ]:
# input the data units
import astropy.units as u
unit = u.electron / u.s
phot = aperture_photometry(data, aperture, error=error, unit=unit)
phot

## Perform aperture photometry at 3 positions
## Also input the error array

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.
apertures = CircularAperture(positions, r=radius)

phot = aperture_photometry(data, apertures, error=error, unit=unit)
phot

In [ ]:
# this time include the Poisson error of the sources
eff_gain = hdr['TEXPTIME']

# this is essentially how the Poission error is included:
#toterr = np.sqrt(err**2 + (data / eff_gain))
#phot = aperture_photometry(data, apertures, error=toterr)

phot = aperture_photometry(data, apertures, error=error,
                           effective_gain=eff_gain)
phot

## Bad pixel masking

In [ ]:
# create a bad pixel
data2 = data.copy()
y, x = 59, 91
data2[y, x] = 100.
aperture_photometry(data2, apertures, error=error)

In [ ]:
# mask the bad pixel
mask = np.zeros_like(data2, dtype=bool)
mask[y, x] = True
aperture_photometry(data2, apertures, error=error, mask=mask)

## Add columns to the photometry table

In [ ]:
# Add an ID column
phot['id'] = np.arange(len(phot)) + 1
phot

In [ ]:
# rearrange the column order to put 'id' first
colnames = phot.colnames
colnames.insert(0, colnames.pop(colnames.index('id')))
phot = phot[colnames]
phot

In [ ]:
# calculate the SNR and add it to the table
snr = phot['aperture_sum'] / phot['aperture_sum_err']
phot['snr'] = snr
phot

In [ ]:
# calculate the F160W AB magnitude and add it to the table
f160w_zpt = 25.9463
abmag = -2.5 * np.log10(phot['aperture_sum']) + f160w_zpt
phot['abmag'] = abmag
phot

In [ ]:
# calculate the ICRS RA and Dec and add them to the table
from astropy.wcs.utils import pixel_to_skycoord
x, y = np.transpose(positions)
coord = pixel_to_skycoord(x, y, wcs)
phot['ra_icrs'] = coord.icrs.ra
phot['dec_icrs'] = coord.icrs.dec
phot

## Exercise 1

Calculate the F160W AB magnitude of the source at $x$=34.61, $y$=137.99 in a $r$=7 pixel (0.42 arcsec) circular aperture.

*Hint*:  The F160W AB zero-point is 25.9463.

In [ ]:
# answer here

## Aperture photometry using Sky apertures

In [ ]:
coord

In [ ]:
# define circular aperture using sky coordinates
from photutils import SkyCircularAperture
radius2 = 5. * u.pix   # sky aperture radius must be a Quantity
sky_apers = SkyCircularAperture(coord, r=radius2)

In [ ]:
# aperture_photometry needs either an wcs object
phot2 = aperture_photometry(data, sky_apers, wcs=wcs)
phot2

In [ ]:
# ...or an hdu (i.e. header and data)
sci_hdulist[0].header['BUNIT'] = 'electron/s'
phot2 = aperture_photometry(sci_hdulist[0], sky_apers)
phot2

In [ ]:
# SkyApertures can also take angular radii
radius3 = 5. * u.arcsec
sky_apers3 = SkyCircularAperture(coord, r=radius3)
sky_apers3.r

In [ ]:
phot3 = aperture_photometry(data, sky_apers3, wcs=wcs)
phot3

## Apertures can plot themselves

In [ ]:
plt.imshow(scale_image(data, scale='sqrt', percent=99.))
apertures.plot(color='blue')   # r=5 pix apertures

## Multiple Apertures at Each Position

In [ ]:
unit = u.electron / u.s
radii = [3.1, 4.2, 5.1]   # pixels
tbl_list = []
for radius in radii:
    tbl_list.append(aperture_photometry(data,
        CircularAperture(positions, radius),
        error=error, unit=unit))

In [ ]:
# combine the table list using hstack
# instead of 1, 2, 3 names (default), use r3, r4, r5
from astropy.table import hstack
names = ['r3', 'r4', 'r5']
phot2 = hstack(tbl_list, table_names=names)
phot2

In [ ]:
# do some table cleanup
for key in ['xcenter_r4', 'ycenter_r4', 'xcenter_r5', 'ycenter_r5']:
    del phot2[key]
phot2.rename_column('xcenter_r3', 'xcenter')
phot2.rename_column('ycenter_r3', 'ycenter')
colnames = phot2.colnames
colnames.insert(0, colnames.pop(colnames.index('ycenter')))
colnames.insert(0, colnames.pop(colnames.index('xcenter')))
phot2 = phot2[colnames]
phot2

In [ ]:
# add the aperture radii to the table
phot2['aperture_radius_r3'] = radii[0] * u.pix
phot2['aperture_radius_r4'] = radii[1] * u.pix
phot2['aperture_radius_r5'] = radii[2] * u.pix
phot2

## Encircled flux

In [ ]:
# multiple apertures at a single position
radii = np.linspace(0.1, 20, 100)
flux = []
for r in radii:
    ap = CircularAperture(positions[0], r=r)
    phot = aperture_photometry(data, ap)
    flux.append(phot['aperture_sum'][0])

In [ ]:
plt.plot(radii, flux)
plt.title('Encircled Flux')
plt.xlabel('Radius (pixels)')
plt.ylabel('Aperture Sum ($e^{-1}/s$)')

## Local background estimation

In [ ]:
# define a circular aperture
apertures = CircularAperture(positions, r=3)

In [ ]:
# and a circular annulus aperture
from photutils import CircularAnnulus
bkg_apertures = CircularAnnulus(positions, r_in=10., r_out=15.)

In [ ]:
# plot the apertures
from astropy.visualization import scale_image
plt.imshow(scale_image(data, scale='sqrt', percent=98.), 
           origin='lower')
apertures.plot(color='blue')
bkg_apertures.plot(color='cyan', hatch='//', alpha=0.8)

In [ ]:
# measure the aperture sums in both apertures
phot = aperture_photometry(data, apertures)
bkg = aperture_photometry(data, bkg_apertures)

In [ ]:
# calculate the mean background level (per pixel) in the annuli
bkg_mean = bkg['aperture_sum'] / bkg_apertures.area()
bkg_mean

In [ ]:
# now calculate the total background in the circular aperture
bkg_sum = bkg_mean * apertures.area()

phot['bkg_sum'] = bkg_sum
phot

In [ ]:
 # subtract the background
flux_bkgsub = phot['aperture_sum'] - bkg_sum

phot['aperture_sum_bkgsub'] = flux_bkgsub
phot

## Part 2:  Image Segmentation

In [ ]:
# define a threshold image
# 2-sigma (per pixel!) above the background
bkg = 0.
threshold = bkg + (2.0 * error)

In [ ]:
# detect 8-connected sources of size 5 pixels
# where each pixel is 2 sigma above the background
from photutils import detect_sources
npixels = 5
segm = detect_sources(data, threshold, npixels)

In [ ]:
# the minimum total object snr is:
obj_min_snr = np.sqrt(5) * 2.
obj_min_snr

### The segmentation image is the isophotal footprint of each source above the threshold

In [ ]:
# plot the segmentation image
from photutils.utils import random_cmap
rand_cmap = random_cmap(random_state=12345)

plt.imshow(segm, cmap=rand_cmap)
print('Found {0} sources'.format(segm.max))

## Filter (smooth) the data prior to source detection

In [ ]:
# define the kernel
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
sigma = 2.0 * gaussian_fwhm_to_sigma    # FWHM = 2 pixels
kernel = Gaussian2DKernel(sigma, x_size=5, y_size=5)
kernel.normalize()
kernel.array

In [ ]:
# detect the sources
segm = detect_sources(data, threshold, npixels, filter_kernel=kernel)
plt.imshow(segm, cmap=rand_cmap)
print('Found {0} sources'.format(segm.max))

## Source deblending

In [ ]:
from photutils import deblend_sources
segm2 = deblend_sources(data, segm, npixels, filter_kernel=kernel,
                        contrast=0.001, nlevels=32)
plt.imshow(segm2, cmap=rand_cmap)
print('Found {0} sources'.format(segm2.max))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
ax1.imshow(scale_image(data, scale='sqrt', percent=99.5))
ax2.imshow(segm2, cmap=rand_cmap)

## Measure the photometry and morphological properties of detected sources

In [ ]:
from photutils import source_properties, properties_table
props = source_properties(data, segm2.data, error=error, wcs=wcs)

`props` is a list of `SourceProperties` objects.

We can create a Table of isophotal photometry and morphological properties using the ``properties_table()`` function:

In [ ]:
tbl = properties_table(props)
tbl

In [ ]:
# a subset of sources can be specified,
# defined by their labels in the segmentation image
labels = [1, 5, 7, 12]
props2 = source_properties(data, segm.data, error=error, wcs=wcs, labels=labels)
tbl = properties_table(props2)
tbl

In [ ]:
# a subset of columns can be specified
columns = ['id', 'xcentroid', 'ycentroid', 'source_sum', 'area']
tbl2 = properties_table(props2, columns=columns)
tbl2

Some additional source properties, e.g. image cutouts, can be accessed directly via the `SourceProperties` objects.

In [ ]:
# get a single object (id=12)
obj = props[11]
obj.id

In [ ]:
# plot its cutout data and error images
fig, ax = plt.subplots(figsize=(4, 6), ncols=2)
cmap = 'hot'
ax[0].imshow(obj.data_cutout_ma, cmap=cmap)
ax[1].imshow(obj.error_cutout_ma, cmap=cmap)

In [ ]:
f160w_zpt = 25.9463
abmag = -2.5 * np.log10(obj.source_sum) + f160w_zpt
print('AB magnitude: {0:0.2f}'.format(abmag))

snr = obj.source_sum / obj.source_sum_err
print('Isophotal SNR: {0:0.2f}'.format(snr))

Please see the complete list of available [source properties](http://photutils.readthedocs.org/en/latest/api/photutils.segmentation.SourceProperties.html#photutils.segmentation.SourceProperties).

## Define the approximate isophotal ellipses for each object

Create elliptical apertures for each object using the measured morphological parameters.

In [ ]:
from photutils import EllipticalAperture
r = 3.    # approximate isophotal extent
apertures = []
for prop in props:
    position = (prop.xcentroid.value, prop.ycentroid.value)
    a = prop.semimajor_axis_sigma.value * r
    b = prop.semiminor_axis_sigma.value * r
    theta = prop.orientation.value
    apertures.append(EllipticalAperture(position, a, b, theta=theta))

Now plot the elliptical apertures on the data and segmentation image.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 8))
ax1.imshow(scale_image(data, scale='sqrt', percent=98.))
ax2.imshow(segm2, cmap=rand_cmap)
for aperture in apertures:
    aperture.plot(color='red', lw=1.5, alpha=0.5, ax=ax1)
    aperture.plot(color='white', lw=1.5, alpha=1.0, ax=ax2)

The segmentation image can be reused on other data:

* e.g. define the segmenation image from a detection image then use it to perform photometry in individual bands
* the data must be registered pixelwise to the detection image


In [ ]:
# read the XDF WFC3/IR F105W data
sci_fn = 'data/xdf_hst_wfc3ir_60mas_f105w_sci.fits'
err_fn = 'data/xdf_hst_wfc3ir_60mas_f105w_rms.fits'
data_f105w = fits.getdata(sci_fn)
error_f105w = fits.getdata(err_fn)

In [ ]:
# extract and measure sources in the F105W data
props_f105w = source_properties(data_f105w, segm2.data, error=error_f105w, wcs=wcs, labels=labels)
columns = ['source_sum', 'source_sum_err']
tbl_f105w = properties_table(props_f105w, columns=columns)
tbl_f105w

In [ ]:
# combine the F160W and F105W tables
from astropy.table import hstack
phot = hstack([tbl, tbl_f105w], table_names=['f160w', 'f105w'])
phot

## The segmentation image can also be modified before measuring photometry/properties, e.g.:

 - remove source segments (artifacts, diffraction spikes, etc.)
 - combine segments
 - mask regions of segmentation image (e.g. near image borders)

See [modifying segmentation images](http://photutils.readthedocs.org/en/latest/photutils/segmentation.html#modifying-segmentation-images) for futher information.
 
### A SExtractor segmentation image can be input to source_properties()
```
CHECKIMAGE_TYPE   SEGMENTATION
CHECKIMAGE_NAME   segmentation.fits
```

## Part 3: Detecting Stars (DAOFIND)

In [ ]:
# load a star image
# https://github.com/astropy/photutils-datasets/blob/master/data/M6707HH.fits.gz
from photutils.datasets import load_star_image
hdu = load_star_image()    
star_data = hdu.data[0:400, 0:400]

In [ ]:
# estimate background and background rms from
# sigma-clipped statistics
from astropy.stats import sigma_clipped_stats
mean, median, std = sigma_clipped_stats(star_data, sigma=3.0)

In [ ]:
# find stars using daofind
from photutils import daofind
sources = daofind(star_data - median, fwhm=3.0, threshold=5.*std)    
sources

In [ ]:
# plot the results
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
positions = (sources['xcentroid'], sources['ycentroid'])
apertures = CircularAperture(positions, r=4.)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(star_data, cmap='Greys', origin='lower', norm=norm)
apertures.plot(color='blue', lw=1.5, alpha=0.5)

## Part 4: 2D Background estimation

In [ ]:
# lets add a large background gradient
ny, nx = star_data.shape
y, x = np.mgrid[:ny, :nx]
gradient =  x * y * 1.e-1
data2 = star_data + gradient
plt.imshow(scale_image(data2, scale='sqrt', percent=99.5),
           cmap='Greys')

In [ ]:
# calculate a 2D background image
from photutils.background import Background
bkg = Background(data2, (10, 10), filter_shape=(3, 3), method='median')

In [ ]:
# plot the estimated background
plt.imshow(bkg.background, cmap='Greys')

In [ ]:
# plot the background-subtracted data
plt.imshow(scale_image(data2 - bkg.background, scale='sqrt', percent=99.5),
           cmap='Greys')

In [ ]:
threshold = median + 2.*std
segm_stars = detect_sources(data2 - bkg.background, threshold, npixels=5)
plt.imshow(segm_stars, cmap=rand_cmap)

In [ ]:
# create a source mask from the segmentation image
mask = (segm_stars.data > 0)
bkg2 = Background(data2, (10, 10), filter_shape=(3, 3),
                  method='sextractor', mask=mask)

In [ ]:
plt.imshow(scale_image(data2 - bkg2.background, scale='sqrt', percent=99.5),
           cmap='Greys')

In [ ]:
y0 = 200
data2_sub = data2 - bkg2.background
plt.plot(star_data[y0, :])
plt.plot(data2[y0, :])
plt.plot(data2_sub[y0, :], color='cyan')
plt.plot(star_data[y0, :] - median)
plt.axhline(0.0)